In [1]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import nltk
# nltk.download('wordnet')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from joblib import parallel_backend
from dask.distributed import Client
import joblib
from sklearn.preprocessing import FunctionTransformer
from functools import partial
import scipy as sp
import math, re
import numpy as np
import csv
import concurrent.futures as cf
import matplotlib.pyplot as plt
from collections import defaultdict
# from numba import jit

pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
netflix_df = pd.read_csv("netflix.csv")
netflix_df.shape

(693, 39)

In [3]:
import string

def tokenize_snow(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)
        
def tokenize_lemma(text):
    stem = nltk.wordnet.WordNetLemmatizer()
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)
        
def tokenize_stem(text):
    stem = nltk.stem.PorterStemmer()
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)

In [4]:
# @jit(nopython=True)
def jsd(p, q, base=np.e): # JS distance between probability vectors, used to compute compH
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    p = np.asarray(p)
    q = np.asarray(q)
    m = 1 / 2 * (p + q)
    
    return sp.stats.entropy(p, m, base=base) / 2 +  sp.stats.entropy(q, m, base=base) / 2

In [5]:
# @jit(nopython=True)
def conth(p_mtx_df): # function to measure content heterogeneity given a topic (prob) matrix
    '''
    How is this review spread across the topics.
    Then you take the average values across the reviews
    Herfindall index
    Assuming the reviews are about culture
    Are people on average focus on a few cultural values (topics) when they write their review
    '''
    return (1 / ((sum(map(sum, np.square(p_mtx_df.values)))) / p_mtx_df.shape[0]))

Needs to have an additional argument. Or be completely redisigned

In [35]:
def comph(probMatrix_df, arr_or_df='df'): 

    if arr_or_df == 'df':
        probMatrix = probMatrix_df
    else:
        probMatrix = probMatrix_df.values
    
    df = pd.DataFrame()
    for x in range(len(probMatrix)): 
        jsd_list = []
        for y in range(len(probMatrix)): 
            jsd_list.append(jsd(probMatrix[x], probMatrix[y]))
        df[str(x)] = jsd_list

    #Get df lower diagonal
    mask = np.ones(df.shape, dtype='bool')
    mask[np.triu_indices(len(df))] = False
    df_lower_diagonal = df[(df>-1) & mask]
    
    distance_list = []
    for k in df.columns: 
    #Transform each column of df_lower_diagonal into list
        col_array = df_lower_diagonal.loc[df_lower_diagonal[k].notna(), k].values
        for d in col_array:
            distance_list.append(d)

    return (sum(distance_list) / len(distance_list))

In [8]:
def comp_name_out(data, col_to_search, col_reviews, companies_list):
    
    """
    This function takes in a dataframe, the name of the column with all of 
    the companies, the name of the column with the reviews, and an iterable
    with the companies names that are in the dataset. The latter could be a list,
    set, Series, tuple, etc.
    """
    
    for company in companies_list:
        condition = (data[col_to_search] == company)
        data.loc[condition, col_reviews] = data.loc[condition, col_reviews].str.lower().str.strip(company.lower())
    
    return data

In [9]:
comp_list = ['Netflix', 'amazon'] 
netflix_df = comp_name_out(netflix_df, 'employerName', 'pros', comp_list)

In [10]:
data_pros = netflix_df['pros'].values
data_pros[0]

'you will be working with the most talented ppl around.'

In [11]:
def normalize_doc(doc):
    """
    This function normalizes your list of documents by taking only
    words, numbers, and spaces in between them. It then filters out
    stop words if you want to.
    """
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens]
#     filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

corp_normalizer = np.vectorize(normalize_doc)

In [12]:
with cf.ProcessPoolExecutor() as executor:
    data_pros_cleaned = executor.map(normalize_doc, data_pros)
    data_pros_cleaned = list(data_pros_cleaned)

TotalWords_vectorizer = CountVectorizer(stop_words='english')
TotalWords_tf = TotalWords_vectorizer.fit_transform(data_pros_cleaned)
totWords = len(TotalWords_vectorizer.get_feature_names())
# totWords


tf_vectorizer = CountVectorizer(tokenizer=tokenize_snow, max_df = 0.90, min_df=0.01)

tf = tf_vectorizer.fit_transform(data_pros_cleaned)

tf_feature_names = tf_vectorizer.get_feature_names()

percVoc = len(tf_feature_names) / totWords * 100
percVoc

16.52046783625731

In [ ]:
%%time

i = 0    
output=np.zeros((60,3))

#totWordsPerdocument = np.sum(tf_matrix, axis=1)
for topics in range(2,300,5): 
    
    lda = LatentDirichletAllocation(n_components=topics, max_iter=200, 
                                    learning_method='online',random_state=1234, n_jobs=-1)
    lda_fit = lda.fit(tf)
    #output normalized matrix with distributions of topics over words
    #normalized
    topicsOverWords = lda_fit.components_ / lda_fit.components_.sum(axis=1)[:, np.newaxis]
    topicsDissim_avg = comph(topicsOverWords)

#store results per firm   
    output[i,0] = topics
    output[i,1] = topicsDissim_avg 
    output[i,2] = percVoc
  
    i = i+1
    
    print(f'Done with topic {topics}')

Done with topic 2
Done with topic 7
Done with topic 12
Done with topic 17
Done with topic 22
Done with topic 27
Done with topic 32
Done with topic 37
Done with topic 42
Done with topic 47
Done with topic 52
Done with topic 57
Done with topic 62
Done with topic 67
Done with topic 72
Done with topic 77
Done with topic 82
Done with topic 87
Done with topic 92
Done with topic 97
Done with topic 102
Done with topic 107
Done with topic 112
Done with topic 117
Done with topic 122
Done with topic 127
Done with topic 132
Done with topic 137
Done with topic 142
Done with topic 147
Done with topic 152
Done with topic 157
Done with topic 162
Done with topic 167
Done with topic 172
Done with topic 177
Done with topic 182
Done with topic 187
Done with topic 192
Done with topic 197
Done with topic 202
Done with topic 207
Done with topic 212
Done with topic 217
Done with topic 222
Done with topic 227
Done with topic 232
Done with topic 237
Done with topic 242
Done with topic 247
Done with topic 252
Do

In [ ]:
%%time


output = defaultdict(np.float32)

for i, topics in enumerate(range(2, 300, 5)):

    lda = LatentDirichletAllocation(n_components=topics, max_iter=200, learning_method='online', 
                                    random_state=1234, n_jobs=-1)
    ldamo = lda.fit(tf)

    #output normalized matrix with distributions of topics over words
    #normalized
    topicsOverWords = ldamo.components_ / ldamo.components_.sum(axis=1)[:, np.newaxis]
    topicsDissim_avg = comph(topicsOverWords)

    #store results per firm   
    output[i] = (topics, topicsDissim_avg, percVoc)
    print(f'Done with topic {topics}')

In [ ]:
%%time

# to use joblib with dask we need a client
# client = Client(processes=False)
mo_list = []


for topics in range(2, 300, 5):
    with joblib.parallel_backend('dask'):
        lda = LatentDirichletAllocation(n_components=topics, max_iter=200, learning_method='batch', 
                                        learning_offset=10., evaluate_every=2, random_state=1234)#, n_jobs=-1)
        ldamo = lda.fit(tf)
        mo_list.append((topics, ldamo))

In [79]:
tops = (ldamo.components_ / ldamo.components_.sum(axis=1)[:, np.newaxis])

In [ ]:
topics_coherence_df = pd.DataFrame(output, columns=['topics', 'coherence', 'voc%'])
topics_coherence_df.head()

In [24]:
optimal_topics = int(topics_coherence_df.loc[topics_coherence_df['coherence'].idxmax(), 'topics'])
optimal_topics

152

In [18]:
lda = LatentDirichletAllocation(n_components=optimal_topics,
                                max_iter=200, 
                                learning_method='batch', 
                                learning_offset=10.,
                                evaluate_every=2,
                                random_state=1234,
                                n_jobs=-1)

In [19]:
#generate matrix summarizing distribution of docs (reviews) over topics
probMatrix = lda.fit_transform(tf)

In [20]:
probMatrix

array([[6.57894737e-04, 6.57894737e-04, 6.57894737e-04, ...,
        6.57894737e-04, 6.57894737e-04, 6.57894737e-04],
       [2.61069340e-05, 2.61069340e-05, 2.61069340e-05, ...,
        2.61069340e-05, 2.61069340e-05, 2.61069340e-05],
       [1.31578947e-03, 1.31578947e-03, 1.31578947e-03, ...,
        1.31578947e-03, 1.31578947e-03, 1.31578947e-03],
       ...,
       [1.64473684e-03, 1.64473684e-03, 1.64473684e-03, ...,
        1.64473684e-03, 1.64473684e-03, 1.64473684e-03],
       [3.65497076e-04, 3.65497076e-04, 3.65497076e-04, ...,
        3.65497076e-04, 3.65497076e-04, 3.65497076e-04],
       [7.30994152e-04, 7.30994152e-04, 7.30994152e-04, ...,
        7.30994152e-04, 7.30994152e-04, 7.30994152e-04]])

In [21]:
docs_topics_df = pd.DataFrame(data = probMatrix)
docs_topics_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151
0,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.544156,0.000658,0.000658,0.000658,0.000658,0.257160,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.100658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658,0.000658
1,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.627100,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.016918,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.352092,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
2,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.201316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.201316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.0

In [22]:
%%time

comp_H = comph(docs_topics_df, arr_or_df='arr')
comp_H

CPU times: user 22.9 s, sys: 55.4 ms, total: 23 s
Wall time: 23 s


0.6226699724544705

In [23]:
%%time

conT_H = conth(docs_topics_df)
conT_H

CPU times: user 17.5 ms, sys: 244 µs, total: 17.8 ms
Wall time: 17.7 ms


2.1974641852117056

In [ ]:
def ent_avg(probMatrix): 
    import statistics
    entropy_list = []
    for i in range(len(probMatrix)): 
        entropy_list.append(entropy(probMatrix[i]))
    entropy_avg = statistics.mean(entropy_list)
    return entropy_avg    

In [ ]:
# function to compute the cross-entropy of two probability distributions
def cross_entropy(p, q):
    for i in range(len(p)):
        p[i] = p[i]+1e-12
    for i in range(len(q)):
        q[i] = q[i]+1e-12

    return -sum([p[i]*log2(q[i]) for i in range(len(p))])

In [ ]:
# function to compute the average cross-entropy of a matrix
def avg_crossEnt(probMatrix): 
#    NOTE: Cross entropy is not symmetric. 
#    This function takes both cross-entropy(p,q) and cross-entropy(q,p) 
#    into account when computing the avg
    crossEntropy_list = []
    for i in range(len(probMatrix)):
        for j in range(len(probMatrix)): 
            if i != j:
                crossEntropy_list.append(cross_entropy(probMatrix[i], probMatrix[j]))
    crossEntropy_avg = statistics.mean(crossEntropy_list)
    return crossEntropy_avg    